In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from geopy.distance import geodesic
from datetime import datetime, timedelta
import folium
from folium import Popup
from folium.plugins import HeatMap
import seaborn as sns

In [9]:
# Load the datasets from JSON files
user_data = pd.read_json('database_generator/user_data.json')
landlord_data = pd.read_json('database_generator/landlord_data.json')
history_data = pd.read_json('database_generator/history_data.json')
private_parking_data = pd.read_json('database_generator/private_parking.json')
public_parking_data = pd.read_json('database_generator/public_parking.json')

# Show the first few rows to verify the data
print(user_data.head())
print(landlord_data.head())
print(history_data.head())
print(private_parking_data.head())
print(public_parking_data.head())


    user_id              name       username  password license_plate_number  \
0  UID80994     Mia Schneider  mschneider372  PgGJ1zWU           FR BT 3750   
1  UID72463     Sophia Wagner     swagner382  9IbfpS2W           FR XM 9798   
2  UID78439  Moritz Schneider  mschneider942  IcrXM52q           FR IC 9583   
3  UID99228      Mia Hoffmann   mhoffmann645  2K6P5TM4           FR TZ 3823   
4  UID25562       Anna Müller     amüller269  WiY95whH           FR IZ 2666   

   credits                      email     phone_number  \
0   164.38     mschneider372@mail.com  +49 153 5695266   
1   162.00     swagner382@example.com  +49 157 1251081   
2    46.22  mschneider942@webmail.org  +49 155 3554367   
3   109.71   mhoffmann645@example.com  +49 155 1642871   
4   448.06     amüller269@example.com  +49 159 9889936   

                    address  
0  21 Kirchplatz, Offenburg  
1         52 Hauptstr., Ulm  
2   48 Gartenweg, Stuttgart  
3   90 Gartenweg, Karlsruhe  
4  71 Bahnhofstr., Konstan

In [15]:
# Make sure datetime columns are in datetime format
history_data['datetime'] = pd.to_datetime(history_data['datetime'])

# If any NaN values are present, handle them (drop or fill)
history_data = history_data.dropna(subset=['datetime'])

# Get today's date and the date 30 days ago
time_range = datetime.now() - timedelta(days=1)

# Filter the history_data to include only the rows from the last 30 days
history_data_recent = history_data[history_data['datetime'] >= time_range]

# Function to calculate distance
def calculate_distance(row):
    try:
        parking_location = (row['parking_latitude'], row['parking_longitude'])
        booking_location = (row['booking_latitude'], row['booking_longitude'])
        return geodesic(parking_location, booking_location).km
    except Exception as e:
        print(f"Error calculating distance for row: {row}, Error: {e}")
        return np.nan  # Return NaN for problematic rows

# Filter out rows where any required latitude/longitude values are missing
history_data_recent = history_data.dropna(subset=['parking_latitude', 'parking_longitude', 'booking_latitude', 'booking_longitude'])

# Recalculate distance
history_data_recent['calculated_distance'] = history_data_recent.apply(calculate_distance, axis=1)

In [11]:
# # Create map centered around average location
# map_center = [history_data_recent['booking_latitude'].mean(), history_data_recent['booking_longitude'].mean()]
# m = folium.Map(location=map_center, zoom_start=10)

# # Add markers and polyline
# for _, row in history_data_recent.iterrows():
#     parking_location = (row['parking_latitude'], row['parking_longitude'])
#     booking_location = (row['booking_latitude'], row['booking_longitude'])
#     time_booked = row['datetime']  
#     rating = row['rating']
#     review_text = row['review_text']
#     distance = row['calculated_distance']

#     popup_content = f"""
#     <b>Booking Location:</b> {booking_location}<br>
#     <b>Rating:</b> {rating}<br>
#     <b>Review:</b> {review_text}<br>
#     <b>Time Booked:</b> {time_booked}
#     """
    
#     # Add markers for parking and booking locations
#     folium.Marker(
#         parking_location,
#         popup=Popup(f"Parking Location: {parking_location}", parse_html=True),
#         icon=folium.Icon(color='blue', icon='car', prefix='fa')
#     ).add_to(m)

#     folium.Marker(
#         booking_location,
#         popup=Popup(popup_content, parse_html=True),
#         icon=folium.Icon(color='green', icon='home', prefix='fa')
#     ).add_to(m)
    
#     # Draw a line between the locations
#     folium.PolyLine(
#         [parking_location, booking_location],
#         color="red",
#         weight=2.5,
#         opacity=0.8,
#         tooltip=f"Distance: {distance:.2f} km"
#     ).add_to(m)

# # # Save map to HTML file
# # m.save('map_visualization_recent_days.html')


In [12]:
# import folium
# from folium.plugins import MarkerCluster
# from IPython.display import display

# # Create map centered around average booking location
# map_center = [history_data_recent['booking_latitude'].mean(), history_data_recent['booking_longitude'].mean()]
# m = folium.Map(location=map_center, zoom_start=10)

# # Create a marker cluster for better visualization
# marker_cluster = MarkerCluster().add_to(m)

# # Create a dictionary to map user_id to names
# user_dict = {user['user_id']: user['name'] for user in user_data}

# # Create a dictionary to map parking_id to parking names
# parking_dict = {
#     **{p['private_id']: p['name'] for p in private_parking_freiburg}, 
#     **{p['public_id']: p['name'] for p in public_parking}
# }

# # Add markers and polylines
# for _, row in history_data_recent.iterrows():
#     parking_location = (row['parking_latitude'], row['parking_longitude'])
#     booking_location = (row['booking_latitude'], row['booking_longitude'])
#     time_booked = row['datetime']
#     rating = row['rating']
#     review_text = row['review_text']
#     distance = row['calculated_distance']

#     # Retrieve user name and parking place name
#     user_name = user_dict.get(row['user_id'], "Unknown User")
#     parking_name = parking_dict.get(row['parking_id'], "Unknown Parking")

#     # Popup content
#     popup_content = f"""
#     <b>User:</b> {user_name}  
#     <b>Parking Place:</b> {parking_name}  
#     <b>Rating:</b> {rating}  
#     <b>Review:</b> {review_text}  
#     <b>Time Booked:</b> {time_booked}
#     """

#     # Add markers for parking and booking locations
#     folium.Marker(
#         parking_location,
#         popup=folium.Popup(f"Parking: {parking_name}", parse_html=True),
#         icon=folium.Icon(color='blue', icon='car', prefix='fa')
#     ).add_to(marker_cluster)

#     folium.Marker(
#         booking_location,
#         popup=folium.Popup(popup_content, parse_html=True),
#         icon=folium.Icon(color='green', icon='home', prefix='fa')
#     ).add_to(marker_cluster)

#     # Draw a line between the locations
#     folium.PolyLine(
#         [parking_location, booking_location],
#         color="red",
#         weight=2.5,
#         opacity=0.8,
#         tooltip=f"Distance: {distance:.2f} km"
#     ).add_to(m)


In [13]:
# from IPython.display import display

# # Display the map directly in the notebook
# display(m)

In [14]:
# Create a map centered around a midpoint (average latitude and longitude of parking locations)
map_center = [private_parking_data['latitude'].mean(), private_parking_data['longitude'].mean()]
parking_map = folium.Map(location=map_center, zoom_start=13)

# Combine private and public parking data for the heatmap
combined_parking_data = pd.concat([private_parking_data[['latitude', 'longitude']], 
                                   public_parking_data[['latitude', 'longitude']]], ignore_index=True)

# Add a heatmap layer
HeatMap(combined_parking_data).add_to(parking_map)

# Display the map
parking_map
